In [ ]:
# script to extract data from g adventures API


In [ ]:
# do i need the full name for itinerary n

In [ ]:
# get promotional rates including promotion e.g. 25% of

In [ ]:
# generate a list of possible countries

In [1]:
from gapipy import Client
from datetime import timedelta
import pandas as pd

In [2]:
import re
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access environment variables
api_key = os.getenv("G_ADVENTURES_KEY")

client = Client(application_key=api_key) 

In [3]:
filtered_departures = client.departures.filter(start_date="2024-04-19")

In [4]:
# filtered_departures = client.departures.filter(start_date__gt="2024-04-01", finish_date__lt="2024-04-30")

In [5]:
filtered_departures.count()

144

In [6]:
my_generator = filtered_departures.all() # .all()

In [8]:
departures_dictionaries_list = []
countries_set = set()
reference_count = 0
for value in my_generator:
    reference_count += 1
    if reference_count % 100 == 0:
        print(reference_count)
    try:
        if value.availability['status'] == "AVAILABLE" and value.tour_dossier != None: 
            trip_list = [category['name'] for category in value.tour_dossier.categories if category['category_type']['label'] ==  "Trip Type"]
            if 'Small Group' in trip_list:
                for i in range(len(value.structured_itineraries)):

                    # get tour name
                    tour_name = value.tour_dossier.name

                    # structured itinerary id
                    structured_itineraries_name = value.structured_itineraries[i].name

                    # get the list of countries visited
                    visited_countries_list = []
                    for country in value.tour_dossier.geography['visited_countries']:
                        visited_countries_list.append(country["name"])
                        countries_set.add(country["name"]) #  add countries to set

                    
                    # get start date
                    start_date = value.start_date
                
                    # get duration
                    duration = value.finish_date - value.start_date + timedelta(days=1)
                    duration = duration.days

                    # get pricing form room categories
                    room_categories_dictionaries = {}
                    
                    for room in value.rooms:
                        data = room.to_dict()
                        for price_brand in data["price_bands"]:
                            for price in price_brand["prices"]:
                                if price['currency'] == "GBP":               
                                    if f'{data["name"]} - {price_brand["name"]}' in room_categories_dictionaries:
                                        room_categories_dictionaries[f'{data["name"]} - {price_brand["name"]}'] += price['amount']
                                    else:
                                        room_categories_dictionaries[f'{data["name"]} - {price_brand["name"]}'] = price['amount']

                    # get tour url 
                    tour_url = value.tour_dossier.site_links[0]['href']
                    
                    # get itinerary day by day
                    itinerary = ""
                    itinerary_days = value.structured_itineraries[i]
                    for day in itinerary_days.days:
                        itinerary += f'Day {day.day}: {day.summary}\n'

                    # get tour categories
                    categories_id_list = [i['id'] for i in value.tour_dossier.categories]

                    tour_categories_dictionaries = {}

                    for category in categories_id_list:
                        category_dict = client.tour_categories.get(category).to_dict()
                        category_value = None
                        if category_dict['description'] == None:
                            category_value = category_dict['name']
                        else:
                            category_value = f"{category_dict['name']} - {category_dict['description']}"

                        if category_dict['category_type']['name'] in tour_categories_dictionaries:
                            tour_categories_dictionaries[category_dict['category_type']['name']] += ", " + category_value
                        else:
                            tour_categories_dictionaries[category_dict['category_type']['name']] = category_value

                    # create dictionary
                    dataframe_dictionary = {}
                    dataframe_dictionary['tour_name'] = tour_name
                    dataframe_dictionary['itinerary_name'] = structured_itineraries_name
                    dataframe_dictionary['visited_countries'] = ", ".join(visited_countries_list)
                    dataframe_dictionary['start_date'] = start_date
                    dataframe_dictionary['duration'] = duration
                    dataframe_dictionary['url'] = tour_url
                    dataframe_dictionary['itinerary'] = itinerary


                    # combine dictionaries
                    dataframe_dictionary = dict(dataframe_dictionary, **tour_categories_dictionaries, **room_categories_dictionaries)


                    
                    # add full dict to list
                    departures_dictionaries_list.append(dataframe_dictionary)
                    

                    for category in categories_id_list:
                        category_dict = client.tour_categories.get(category).to_dict()
                        itinerary += f"{category_dict['category_type']['name']}: {category_dict['name']} - {category_dict['description']}\n"

     
    except:
        print("403 issue")


403 issue
403 issue
403 issue
403 issue
100
403 issue
403 issue


In [ ]:
# df = pd.DataFrame(departures_dictionaries_list)
# df.to_csv('raw_data/one_day_test_v2.csv', index=False)
# df

In [11]:
df = pd.DataFrame(departures_dictionaries_list)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   tour_name                      61 non-null     object
 1   itinerary_name                 61 non-null     object
 2   visited_countries              61 non-null     object
 3   start_date                     61 non-null     object
 4   duration                       61 non-null     int64 
 5   url                            61 non-null     object
 6   itinerary                      61 non-null     object
 7   Travel Style                   61 non-null     object
 8   Service Level                  61 non-null     object
 9   Physical Grading               61 non-null     object
 10  Merchandising                  53 non-null     object
 11  Trip Type                      61 non-null     object
 12  Standard - Adult               60 non-null     object
 13  Standar

In [13]:
df.head(3)

,tour_name,itinerary_name,visited_countries,start_date,duration,url,itinerary,Travel Style,Service Level,Physical Grading,Merchandising,Trip Type,Standard - Adult,Standard - Double - D - Adult,Standard - Twin - D - Adult
0,Laos: Sunrises & Street Food,,"Laos, Thailand",2024-04-19,9,https://www.gadventures.com/trips/budget-laos-...,Day 1: Arrive at any time and welcome to Vient...,"18-to-Thirtysomethings - Fast, fresh, and fun ...",Basic - Simple and clean hotels and hostels; a...,2 - Light - Light walking and hiking suitable ...,Planeterra Project,Small Group - Group trips average 12 traveller...,779.00,NaN,NaN
1,Laos to Thailand: Night Markets & Mekong Cruising,,"Laos, Thailand",2024-04-19,11,https://www.gadventures.com/trips/budget-tour-...,Day 1: Arrive at any time and welcome to Vient...,"18-to-Thirtysomethings - Fast, fresh, and fun ...",Basic - Simple and clean hotels and hostels; a...,2 - Light - Light walking and hiking suitable ...,Planeterra Project,Small Group - Group trips average 12 traveller...,799.00,NaN,NaN
2,Laos to Northern Thailand: Treks & Trails,,"Laos, Thailand",2024-04-19,14,https://www.gadventures.com/trips/trekking-lao...,Day 1: Arrive at any time and welcome to Vient...,"18-to-Thirtysomethings - Fast, fresh, and fun ...",Basic - Simple and clean hotels and hostels; a...,3 - Average - Some tours may include light hik...,Planeterra Project,Small Group - Group trips average 12 traveller...,1049.00,NaN,NaN


In [10]:
list(countries_set)

['Chile',
 'Nepal',
 'Brazil',
 'Egypt',
 'South Korea',
 'Maldives',
 'Turkmenistan',
 'Uzbekistan',
 'Thailand',
 'Vietnam',
 'Argentina',
 'Morocco',
 'Peru',
 'Ecuador',
 'Portugal',
 'Indonesia',
 'Spain',
 'Malaysia',
 'India',
 'Laos',
 'Uruguay',
 'Bolivia',
 'Mexico',
 'Cambodia',
 'Jordan',
 'Japan']